In [42]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

In [488]:
data = pd.read_csv('../../proj/data/SS-H.csv')

In [489]:
data.head()

,Width,Complexity,Fifo,Multiplier,Energy-,Inv_runtime-
0,3,1,4,1,7.835103,4.339361
1,3,1,1,1,7.836833,4.309194
2,3,1,2,100,9.965784,5.066154
3,4,0,4,20,7.958239,5.096536
4,4,0,1,20,7.958239,5.096536


In [490]:
#X = data[['Clndrs', 'Volume', 'HpX', 'Model', 'origin']]
#y = data[['Lbs-', 'Acc+', 'Mpg+']]
X_columns = [col for col in data.columns if not (col.endswith('+') or col.endswith('-') or col.endswith('!'))]
y_columns = [col for col in data.columns if col.endswith('+') or col.endswith('-') or col.endswith('!')]

X = data[X_columns]
y = data[y_columns]

In [491]:
X.replace('?', np.nan, inplace=True)

/var/folders/mf/pc621dwd2z3_jmf5p7t1zn2r0000gn/T/ipykernel_47495/2018571216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace('?', np.nan, inplace=True)


In [492]:
column_names = X.columns.tolist()
y_column_names = y.columns.tolist()
features = y_column_names
print(features)

['Energy-', 'Inv_runtime-']


In [493]:
centroids_data = []

In [494]:
#using standard scaler
#X = StandardScaler().fit_transform(X.astype(float))
print(X.shape)

(259, 4)


In [495]:
#using minmax scaler to retain original values
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [496]:
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

In [497]:
#pca = PCA(n_components=2)
#X_pca = pca.fit_transform(X)

In [498]:
dbscan = DBSCAN(eps=0.6, min_samples=3)

In [499]:
dbscan.fit(X)

DBSCAN(eps=0.6, min_samples=3)

In [500]:
labels = dbscan.labels_
data['Cluster'] = labels

In [501]:
unique_labels = np.unique(labels)
#print(f"\t\t     {column_names}")
for label in unique_labels:
    if label == -1:
        continue  # Skip noise points
    cluster_size = np.sum(labels == label)
    cluster_centroid = np.mean(X[labels == label], axis=0)
    #getting back original values to print
    cluster_centroid_original_scale = scaler.inverse_transform([cluster_centroid])
    cluster_centroid_formatted = [f"{value:.2f}" for value in cluster_centroid_original_scale[0]]
    y_centroid = np.mean(y.values[labels == label], axis=0)
    y_centroid_original_scale = y_centroid  # Assuming y values are not scaled
    y_centroid_formatted = [f"{value:.2f}" for value in y_centroid_original_scale]
    
    centroid_data = {'Cluster': label, **dict(zip(column_names, cluster_centroid_formatted)), 
                     **dict(zip(y_column_names, y_centroid_formatted))}
    
    centroids_data.append(centroid_data)
    #print(f"Cluster {label}: Centroid = {cluster_centroid_formatted}, Size = {cluster_size}")
    '''print(f"Cluster {label}:")
    for col, value in zip(column_names, cluster_centroid_formatted):
        print(f"    {col}: {value}")
    print(f"Size = {cluster_size}\n")'''

In [502]:
centroids_df = pd.DataFrame(centroids_data)
print(centroids_df)

   Cluster Width Complexity  Fifo Multiplier Energy- Inv_runtime-
0        0  4.89       1.00  2.42      26.86    8.06         5.00
1        1  4.78       0.00  2.39      26.86    7.75         5.01


In [503]:
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f'Number of clusters formed: {n_clusters}')

Number of clusters formed: 2


In [504]:
centroids_df = centroids_df.apply(pd.to_numeric, errors='ignore')

/var/folders/mf/pc621dwd2z3_jmf5p7t1zn2r0000gn/T/ipykernel_47495/2196313317.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  centroids_df = centroids_df.apply(pd.to_numeric, errors='ignore')


In [505]:
maxValues = data.max()
minValues = data.min()

In [506]:
def normalize_values(min_val, max_val, value):
    return (value - min_val) / (max_val - min_val)

In [507]:
print(centroids_df.min())

Cluster          0.00
Width            4.78
Complexity       0.00
Fifo             2.39
Multiplier      26.86
Energy-          7.75
Inv_runtime-     5.00
dtype: float64


In [508]:
d2h = []
for index, centroid in centroids_df.iterrows():
    d = 0
    for column in features:
        if column.endswith("+"):
            d += (abs(1.0 - normalize_values(minValues[column], maxValues[column], centroid[column])))**2
        else:
            d += (abs(0.0 - normalize_values(minValues[column], maxValues[column], centroid[column])))**2
    d2h.append((d/len(features))**(1/2))

d2h = [round(value, 3) for value in d2h]
centroids_df['d2h'] = d2h

# Print the centroids DataFrame
print(centroids_df)


   Cluster  Width  Complexity  Fifo  Multiplier  Energy-  Inv_runtime-    d2h
0        0   4.89         1.0  2.42       26.86     8.06          5.00  0.697
1        1   4.78         0.0  2.39       26.86     7.75          5.01  0.677


In [509]:
print(min(d2h))

0.677


In [510]:
val = round(min(d2h),2)
val_list = [val] * 20
df = pd.DataFrame([val_list])
print(df)

     0     1     2     3     4     5     6     7     8     9     10    11  \
0  0.68  0.68  0.68  0.68  0.68  0.68  0.68  0.68  0.68  0.68  0.68  0.68   

     12    13    14    15    16    17    18    19  
0  0.68  0.68  0.68  0.68  0.68  0.68  0.68  0.68  


In [511]:
centroids_df.to_csv('Baselines/SS-H-baseline.csv',index=False)

In [391]:
#plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='rainbow')
#plt.xlabel('PC1')
#plt.ylabel('PC2')
#plt.title('dbscan clusters (after pca)')
#plt.show()

In [201]:
core_samples_mask = np.zeros_like(dbscan.labels_, dtype=bool)
core_samples_mask[dbscan.core_sample_indices_] = True
print("Indices of core samples:", dbscan.core_sample_indices_)

Indices of core samples: [  0   2   3   4   5   6   7   8   9  11  12  13  14  15  16  18  19  20
  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  56  58  59
  60  61  62  63  64  65  66  67  68  69  70  73  74  75  76  78  79  80
  82  83  84  86  90  91  92  93  94  95  98  99 100 101 102 103 104 105
 108 109 110 111 112 113 114 115 116 117 119 120 121 123 125 126 127 128
 129 130 131 132 133 135 136 137 138 139 140 142 143 144 145 147 148 149
 151 152 153 155 157 158 159 160 161 162 165 166 167 168 169 170 171 172
 173 174 175 177 178 179 180 182 184 185 186 187 188 189 190 191 194 196
 197 198 199 200 201 202 203 204 205 206 209 211 212 214 215 216 217 218
 219 220 221 222 224 226 227 228 229 230 231 233 234 235 237 238 240 243
 244 245 249 250 251 252 253 254 255 256 257 258 259 260 262 265 266 267
 268 269 270 271 272 273 274 276 277 278 279 280 281 282 283 285 286 287
 288 289 291 294 295 296 2